In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D
from tensorflow.keras.layers import Input, LeakyReLU, Softmax, Reshape, Flatten
from tensorflow.keras.layers import concatenate, maximum, Lambda, Layer
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
import tensorflow.keras.backend as K
import pickle

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Additional layers not available in core Keras

In [2]:
class Bias(Layer):
    """
    Adds bias to a layer. This is used for untied biases convolution. 
    """
    def __init__(self, **kwargs):
        super(Bias, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.bias = self.add_weight(name='bias', 
                                      shape=input_shape[1:],
                                      initializer='uniform',
                                      trainable=True)
        super(Bias, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return tf.add(x, self.bias)

    def compute_output_shape(self, input_shape):
        return input_shape

In [3]:
def feature_pool_max(input, pool_size=2, axis=1):
    """
    Based on lasagne implementation of FeaturePool
    """
    input_shape = input.shape.as_list()
    num_feature_maps = input_shape[axis]
    num_feature_maps_out = num_feature_maps // pool_size
    
    pool_shape = tf.TensorShape(
        (input_shape[1:axis] + [num_feature_maps_out, pool_size] + input_shape[axis+1:])
    )
    
    print(pool_shape)
    input_reshaped = Reshape(pool_shape)(input)
    # reduce along all axis but the target one
    reduction_axis = list(range(1, len(pool_shape)+1))
    reduction_axis.pop(axis-1)
    
    return tf.reduce_max(input_reshaped, axis=reduction_axis)

# create Layer for reshape with batchsize
# strong_reshape_func = lambda x: tf.reshape(x, (batch_size//2, concat.shape[1]*2))
# StrongReshape = Lambda(strong_reshape_func)

# Building the main model

In [4]:
help(Dropout)

Help on class Dropout in module tensorflow.python.keras.layers.core:

class Dropout(tensorflow.python.keras.engine.base_layer.Layer)
 |  Applies Dropout to the input.
 |  
 |  Dropout consists in randomly setting
 |  a fraction `rate` of input units to 0 at each update during training time,
 |  which helps prevent overfitting.
 |  
 |  Arguments:
 |      rate: float between 0 and 1. Fraction of the input units to drop.
 |      noise_shape: 1D integer tensor representing the shape of the
 |          binary dropout mask that will be multiplied with the input.
 |          For instance, if your inputs have shape
 |          `(batch_size, timesteps, features)` and
 |          you want the dropout mask to be the same for all timesteps,
 |          you can use `noise_shape=(batch_size, 1, features)`.
 |      seed: A Python integer to use as random seed.
 |  
 |  Method resolution order:
 |      Dropout
 |      tensorflow.python.keras.engine.base_layer.Layer
 |      tensorflow.python.training.

In [5]:
def build_model(width=512, height=512, filename=None,
                n_classes=5,  p_conv=0.0):
    """
    
    """
    # Input shape (height, width, depth), different from original implimentation
    main_input = Input(shape=(height, width, 3))
    
    # Note: for conv layers paper uses untie_biases=True
    # layer will have separate bias parameters for each position in each channel. 
    # As a result, the b attribute will be a 3D tensor.

    # no need to init weights as they will be loaded from a file
    # Conv layers(filters, kernel_size)
    conv_main_1 = Conv2D(32, 7, strides=(2, 2), padding='same', 
                    use_bias=False,
                    activation= None,
                   )(main_input)
    conv_bias_1 = Bias()(conv_main_1)
    conv_activation_1 = LeakyReLU(alpha=0.5)(conv_bias_1)
    dropout_1 = Dropout(p_conv)(conv_activation_1)
    maxpool_1 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_1)
    # 3
    conv_main_2 = Conv2D(32, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation= None,
                        )(maxpool_1)
    conv_bias_2 = Bias()(conv_main_2)
    conv_activation_2 = LeakyReLU(alpha=0.5)(conv_bias_2)
    dropout_2 = Dropout(p_conv)(conv_activation_2)
    # 4
    conv_main_3 = Conv2D(32, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation= None,
                        )(dropout_2)
    conv_bias_3 = Bias()(conv_main_3)
    conv_activation_3 = LeakyReLU(alpha=0.5)(conv_bias_3)
    dropout_3 = Dropout(p_conv)(conv_activation_3)
    maxpool_3 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_3)
    # 6
    conv_main_4 = Conv2D(64, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(maxpool_3)
    conv_bias_4 = Bias()(conv_main_4)
    conv_activation_4 = LeakyReLU(alpha=0.5)(conv_bias_4)
    dropout_4 = Dropout(p_conv)(conv_activation_4)
    # 7
    conv_main_5 = Conv2D(64, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_4)
    conv_bias_5 = Bias()(conv_main_5)
    conv_activation_5 = LeakyReLU(alpha=0.5)(conv_bias_5)
    dropout_5 = Dropout(p_conv)(conv_activation_5)
    maxpool_5 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_5)
    # 9 
    conv_main_6 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(maxpool_5)
    conv_bias_6 = Bias()(conv_main_6)
    conv_activation_6 = LeakyReLU(alpha=0.5)(conv_bias_6)
    dropout_6 = Dropout(p_conv)(conv_activation_6)
    # 10    
    conv_main_7 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_6)
    conv_bias_7 = Bias()(conv_main_7)
    conv_activation_7 = LeakyReLU(alpha=0.5)(conv_bias_7)
    dropout_7 = Dropout(p_conv)(conv_activation_7)
    # 11
    conv_main_8 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_7)
    conv_bias_8 = Bias()(conv_main_8)
    conv_activation_8 = LeakyReLU(alpha=0.5)(conv_bias_8)
    dropout_8 = Dropout(p_conv)(conv_activation_8)
    # 12
    conv_main_9 = Conv2D(128, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_8)
    conv_bias_9 = Bias()(conv_main_9)
    conv_activation_9 = LeakyReLU(alpha=0.5)(conv_bias_9)
    dropout_9 = Dropout(p_conv)(conv_activation_9)
    maxpool_9 = MaxPool2D(pool_size=3, strides=(2, 2))(dropout_9)
    # 14
    conv_main_10 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(maxpool_9)
    conv_bias_10 = Bias()(conv_main_10)
    conv_activation_10 = LeakyReLU(alpha=0.5)(conv_bias_10)
    dropout_10 = Dropout(p_conv)(conv_activation_10)
    # 15
    conv_main_11 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_10)
    conv_bias_11 = Bias()(conv_main_11)
    conv_activation_11 = LeakyReLU(alpha=0.5)(conv_bias_11)
    dropout_11 = Dropout(p_conv)(conv_activation_11)
    # 16
    conv_main_12 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_11)
    conv_bias_12 = Bias()(conv_main_12)
    conv_activation_12 = LeakyReLU(alpha=0.5)(conv_bias_12)
    dropout_12 = Dropout(p_conv)(conv_activation_12)
    # 17
    conv_main_13 = Conv2D(256, 3, strides=(1, 1), padding='same', 
                         use_bias=False,
                         activation=None,
                        )(dropout_12)
    conv_bias_13 = Bias()(conv_main_13)
    conv_activation_13 = LeakyReLU(alpha=0.5, name='layer_17')(conv_bias_13)
    dropout_13 = Dropout(p_conv)(conv_activation_13)
    maxpool_13 = MaxPool2D(pool_size=3, name="last_conv", strides=(2, 2))(dropout_13)
    # 19, special dropout between phases with p=1/2 
    dropout_inter = Dropout(0.5)(maxpool_13)
    # 20 Dense phase 
    # Begins with Maxout layer, which is implemented here as Dense+custom feature_pool function
    flatten_inter = Flatten()(dropout_inter)
    maxout_1 = Dense(units=1024, 
                     activation=None)(flatten_inter)
    # need to wrap operation in Lambda to count as a layer
    maxout_2 = Lambda(lambda x: feature_pool_max(x, pool_size=2, axis=1))(maxout_1)
    
    # 22 Concatenate with processed img, take both eyes into account
    img_dim_input = Input(shape=(2,), name="imgdim")
    concat = concatenate([maxout_2, img_dim_input], axis=1)
    
    # 24
    # flatten = Reshape((-1, concat.output_shape()[1] * 2))(concat)
    print(concat.shape)
    # use lambda for custom reshape that's capable of changing batch_size as well
    # expect order left-right
    # TODO: (-1, net['23'].output_shape[1] * 2)
    flatten = Lambda(lambda x: K.reshape(x, (-1, concat.shape[1]*2)))(concat)
    dense_droupout_0 = Dropout(0.5)(flatten)
    # 26
    dense_1 = Dense(units=1024,
                    activation=None,
                   )(dense_droupout_0)
    dense_maxpool_1 = Lambda(lambda x: feature_pool_max(x, pool_size=2, axis=1))(dense_1)
    dense_dropout_1 = Dropout(0.5)(dense_maxpool_1)
    
    # 29
    dense_2 = Dense(units=n_classes*2,
                    activation=None,
                   )(dense_dropout_1)
    softmax_flatten = Lambda(lambda x: tf.reshape(x, (-1, n_classes)))(dense_2)
    softmax = keras.layers.Softmax()(softmax_flatten)
    
    model = Model(inputs=[main_input, img_dim_input], outputs=[softmax])
    
    return model

In [6]:
model = build_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(512, 2)
(?, 514)
(512, 2)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 32) 4704        input_1[0][0]                    
__________________________________________________________________________________________________
bias (Bias)                     (None, 256, 256, 32) 2097152     conv2d[0][0]                     
_____________________________________________________________________________________________

## Loading weights

Due to unicode compatability issues between python2 and python3 we use np.save method, with encoding as 'bytes'.

In [7]:
params_names = ['Conv_1_W', 'Conv_1_b', 'Conv_2_W', 'Conv_2_b', 'Conv_3_W', 'Conv_3_b', 
                'Conv_4_W', 'Conv_4_b', 'Conv_5_W', 'Conv_5_b', 'Conv_6_W', 'Conv_6_b', 
                'Conv_7_W', 'Conv_7_b', 'Conv_8_W', 'Conv_8_b', 'Conv_9_W', 'Conv_9_b', 
                'Conv_10_W', 'Conv_10_b', 'Conv_11_W', 'Conv_11_b', 'Conv_12_W', 'Conv_12_b', 
                'Conv_13_W', 'Conv_13_b', 'Dense_1_W', 'Dense_1_b', 'Dense_2_W', 'Dense_2_b', 
                'Dense_3_W', 'Dense_3_b']

In [8]:
arr = np.load("../models/JFnet_weights.npy", encoding="bytes")

In [9]:
layer_to_weights = dict(zip(params_names, arr))
layer_to_weights['Dense_3_b']

array([ 0.5916246 ,  0.04647866,  0.40189323, -0.12301593, -0.41697878,
        0.6376505 ,  0.02411902,  0.393416  , -0.03656115, -0.51861304],
      dtype=float32)

In [11]:
def update_weights(model, layer_to_weights):
    # set all layers in the model
    c_conv = 1
    c_dense = 1
    # lasagne store weights not their transpose, so we transpose
    for layer in model.layers:
        layer_str = str(layer)
        if "Conv2D" in layer_str:
            layer.set_weights([
                layer_to_weights['Conv_{}_W'.format(c_conv)].T
            ])
        if "Bias" in layer_str:
            layer.set_weights([
                layer_to_weights['Conv_{}_b'.format(c_conv)].T
            ])
            c_conv +=1
        if "Dense" in layer_str:
            layer.set_weights([
                layer_to_weights['Dense_{}_W'.format(c_dense)],
                layer_to_weights['Dense_{}_b'.format(c_dense)]
            ])
            c_dense += 1

update_weights(model, layer_to_weights)

# Save models with these weights for future use in the class

In [12]:
model.save_weights("../models/keras_JFnet.h5")

In [13]:
model2 = build_model()

(512, 2)
(?, 514)
(512, 2)


In [14]:
model2.input_shape

[(None, 512, 512, 3), (None, 2)]

In [15]:
model.predict([np.zeros((64, 512, 512, 3)), np.zeros((64, 2))])

array([[0.29261371, 0.26478586, 0.32950336, 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923906],
       [0.29261371, 0.26478586, 0.32950336, 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923906],
       [0.29261371, 0.26478586, 0.32950336, 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923906],
       [0.29261371, 0.26478586, 0.32950336, 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923906],
       [0.29261371, 0.26478586, 0.32950336, 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923906],
       [0.29261371, 0.26478586, 0.32950336, 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923906],
       [0.29261371, 0.26478586, 0.32950336, 0.0867935 , 0.02630354],
       [0.30382672, 0.26224723, 0.3291842 , 0.08550265, 0.01923906],
       [0.29261371, 0.26478586, 0.

In [16]:
model2.load_weights("../models/keras_JFnet.h5")

## Peeling off layers and BCNN

In [17]:
n_classes=2 

In [18]:
conv_output = model2.get_layer("layer_17").output

mean_pooled = GlobalAveragePooling2D(
    data_format='channels_last')(conv_output)
max_pooled = GlobalMaxPooling2D(
    data_format='channels_last')(conv_output)
global_pool = concatenate([mean_pooled, max_pooled], axis=1)

softmax_input = Dense(
    units=n_classes, activation=None,)(global_pool)
softmax_output = Softmax()(softmax_input)

bcnn_model = Model(inputs=[model2.input[0]], outputs=[softmax_output])

In [19]:
bcnn_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 256, 256, 32) 4704        input_2[0][0]                    
__________________________________________________________________________________________________
bias_13 (Bias)                  (None, 256, 256, 32) 2097152     conv2d_13[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_12 (LeakyReLU)      (None, 256, 256, 32) 0           bias_13[0][0]                    
__________________________________________________________________________________________________
dropout_16

## Saving bcnn models

In [20]:
bcnn_param_names = ['Conv_1_W', 'Conv_1_b', 'Conv_2_W', 'Conv_2_b', 'Conv_3_W', 'Conv_3_b', 
                    'Conv_4_W', 'Conv_4_b', 'Conv_5_W', 'Conv_5_b', 'Conv_6_W', 'Conv_6_b', 
                    'Conv_7_W', 'Conv_7_b', 'Conv_8_W', 'Conv_8_b', 'Conv_9_W', 'Conv_9_b', 
                    'Conv_10_W', 'Conv_10_b', 'Conv_11_W', 'Conv_11_b', 'Conv_12_W', 'Conv_12_b', 
                    'Conv_13_W', 'Conv_13_b', 'Dense_1_W', 'Dense_1_b']

In [21]:
bcnn_weights = np.load("../models/weights_bcnn1_392bea6.npy", encoding="bytes")

In [22]:
len(bcnn_weights) == len(params_names)

False

In [23]:
layer_to_weights = dict(zip(params_names, bcnn_weights))

In [24]:
list(zip(params_names, bcnn_weights))

[('Conv_1_W', array([[[[-2.02283758e-04, -3.63964657e-03, -2.76703164e-02, ...,
             3.15386243e-02,  8.84557664e-02,  4.42670509e-02],
           [ 6.55262219e-03, -5.91572262e-02, -1.02498494e-01, ...,
             2.23688893e-02,  3.77700180e-02,  4.00122479e-02],
           [-4.10356112e-02, -7.45726898e-02, -9.29462165e-02, ...,
            -1.45968515e-02,  3.44603881e-02,  3.92032228e-02],
           ...,
           [-1.42963389e-02,  3.95306312e-02,  9.91340801e-02, ...,
             6.28545955e-02, -1.63403060e-02,  1.13455998e-02],
           [-1.43537195e-02,  1.44279907e-02,  7.39849433e-02, ...,
             8.91827792e-03, -2.15024762e-02,  4.07190621e-03],
           [ 1.19553506e-02, -1.82888154e-02, -3.33811603e-02, ...,
            -1.66660491e-02, -1.30034871e-02,  7.59386783e-03]],
  
          [[-2.17148177e-02,  3.20617072e-02,  3.76722775e-02, ...,
            -1.65563803e-02, -3.93776745e-02, -7.44751692e-02],
           [ 2.76255440e-02,  3.06528620e-02

In [25]:
update_weights(bcnn_model, layer_to_weights)

In [26]:
bcnn_model.get_weights()

[array([[[[-2.02283758e-04,  9.65800416e-03,  2.91509903e-03, ...,
           -1.92294400e-02, -3.06822732e-02,  2.52537094e-02],
          [-2.17148177e-02,  1.58241123e-01, -3.59797738e-02, ...,
            2.01683994e-02,  8.50977451e-02,  8.63571465e-02],
          [-9.98002216e-02,  2.39440277e-02,  5.08205928e-02, ...,
           -2.64430977e-02,  1.94256771e-02, -3.16414088e-02]],
 
         [[ 6.55262219e-03,  1.49273770e-02,  6.36618817e-03, ...,
           -6.90151239e-03, -1.00459792e-02,  2.35415585e-02],
          [ 2.76255440e-02,  5.25807329e-02, -8.66641998e-02, ...,
            4.17693593e-02,  8.94546881e-02,  1.50681306e-02],
          [-3.28424275e-02,  2.65412815e-02,  2.13785563e-02, ...,
           -4.49424870e-02, -1.72187779e-02,  1.20812692e-02]],
 
         [[-4.10356112e-02, -3.00695859e-02, -1.45654604e-02, ...,
           -2.19050720e-02, -2.36291382e-02,  2.03474308e-03],
          [-2.33415924e-02, -4.99418601e-02, -6.18538670e-02, ...,
            8.240

In [27]:
len(bcnn_model.get_weights())

28

In [28]:
bcnn_model.save_weights("../models/weights_bcnn1_392bea6.h5")

In [29]:
bcnn2_model = tf.keras.models.clone_model(bcnn_model)

In [30]:
bcnn2_model.load_weights("../models/weights_bcnn1_392bea6.h5")

## Prediction with uncertainty

In [34]:
bcnn_model_input = np.zeros(bcnn_model.input_shape[1:])
bcnn_model_input = bcnn_model_input.reshape((-1,) + bcnn_model.input_shape[1:])
model2_input = [np.zeros((64, 512, 512, 3)), np.zeros((64, 2))]

In [35]:
# prediction with uncertainty
f = K.function(
    [bcnn_model.input, K.learning_phase()],
    bcnn_model.layers[-1].output,
)

In [36]:
bcnn_model_input.shape

(1, 512, 512, 3)

In [37]:
f([bcnn_model_input, 1])

array([[0.8102807 , 0.18971938]], dtype=float32)